# 4. Gt, Lt, Between, Within, Mean, Dedup, Where, Neq, Limit, Range, Skip, Tail, TimeLimit
#### By Kwan Yin Andrew Chau

In [1]:
import nbfinder
from loader import g
from loader import P
from loader import __
from loader import hkVertexId
from loader import sydneyVertexId

## gt(), lt()
__Get destinations from HK that have more than 5 runways__

In [2]:
g.V(hkVertexId).out('route').has('runways', P.gt(5)).valueMap('desc', 'runways').fold().next()

[{'runways': [6], 'desc': ['Boston Logan']},
 {'runways': [7], 'desc': ['Dallas/Fort Worth International Airport']},
 {'runways': [8], 'desc': ["Chicago O'Hare International Airport"]},
 {'runways': [6], 'desc': ['Amsterdam Airport Schiphol']}]

## between()
__Get destinations from HK that have 6 to 8 runways__

In [3]:
g.V(hkVertexId).out('route').has('runways', P.between(6, 8)).valueMap('desc', 'runways').fold().next()

[{'runways': [6], 'desc': ['Boston Logan']},
 {'runways': [7], 'desc': ['Dallas/Fort Worth International Airport']},
 {'runways': [6], 'desc': ['Amsterdam Airport Schiphol']}]

## within()
__Count of vertices that are within region "AU-NSW" and "AU-VIC"__

In [4]:
g.V().has('region', P.within('AU-NSW', 'AU-VIC')).values('desc').fold().next()

['Tamworth Airport',
 'Port Macquarie Airport',
 'Lismore Airport',
 'Wagga Wagga City Airport',
 'Moruya Airport',
 'Albury Airport',
 'Taree Airport',
 'Merimbula Airport',
 'Parkes Airport',
 'Coffs Harbour Airport',
 'Narrandera Airport',
 'Griffith Airport',
 'Sydney Kingsford Smith',
 'Portland Airport',
 'Narrabri Airport',
 'Avalon Airport',
 'Melbourne International Airport',
 'Lord Howe Island Airport',
 'Warrnambool Airport',
 'Sydney Bankstown Airport',
 'Bathurst Airport',
 'Orange Airport',
 'Gold Coast Airport',
 'Mildura Airport',
 'Grafton Airport',
 'Melbourne Essendon Airport',
 'Ballina Byron Gateway Airport',
 'Newcastle Airport',
 'Moree Airport',
 'Broken Hill Airport',
 'Dubbo City Regional Airport',
 'Armidale Airport']

## mean()

__Get the mean between the number of runways of airports within regions "AU-NSW" and "AU-VIC"__

In [5]:
g.V().has('region', P.within('AU-NSW', 'AU-VIC')).values('runways').mean().next()

1.875

## dedup()

__Get airports that have destinations the HK airport does, and remove duplicates__

In [6]:
x1 = g.V(hkVertexId).out('route').in_('route').dedup().count().next()
g.V(hkVertexId).out('route').in_('route').values('desc').dedup().fold().next()

['Salt Lake City',
 'Baltimore/Washington International Airport',
 'Anchorage Ted Stevens',
 'Orlando International Airport',
 'Austin Bergstrom International Airport',
 'Boston Logan',
 'Minneapolis-St.Paul International Airport',
 'Hartsfield - Jackson Atlanta International Airport',
 'Nashville International Airport',
 'Las Vegas Mc Carran',
 'George Bush Intercontinental',
 'Toronto Pearson International Airport',
 'Raleigh-Durham',
 'Tampa International Airport',
 'Phoenix Sky Harbor International Airport',
 'Ronald Reagan Washington National Airport',
 'Honolulu International Airport',
 'Los Angeles International Airport',
 'Helsinki Ventaa',
 'London Heathrow',
 'San Antonio',
 'Orange County/Santa Ana, John Wayne',
 'Dallas/Fort Worth International Airport',
 'Dublin International Airport',
 'Beijing Capital International Airport',
 'Detroit Metropolitan, Wayne County',
 'Manchester Airport',
 'Fort Lauderdale/Hollywood International Airport',
 'Hong Kong - Chek Lap Kok Interna

__Remove duplicates by number of runways__

In [7]:
g.V().has('region','AU-NSW').dedup().by('runways').values('code','runways').fold().next()

['TMW', 4, 'PQQ', 2, 'LSY', 1, 'SYD', 3]

__Remove duplicates where there is the same A and C even if B is different__

In [8]:
g.V(hkVertexId).as_('a').V(sydneyVertexId).as_('c').both().as_('b').limit(10).dedup('a','c').select('a','b','c').fold().next()

[{'a': v[24624], 'b': v[4240], 'c': v[94344]}]

## where(), neq()

Exclude the HK airport itself

In [9]:
x2 = g.V(hkVertexId).as_('hk').out('route').in_('route').where(P.neq('hk')).dedup().count().next()
assert x1 == x2 + 1
x2

1879

## limit()
Limit the amount of results to "n"

In [10]:
n = 5
count = g.V(hkVertexId).out('route').limit(n).count().next()
assert n == count
x1 = g.V(hkVertexId).out('route').limit(n).fold().next()
x1

[v[4240], v[4312], v[12320], v[12504], v[16392]]

## range()
Equivalent to using a range starting from 0

In [11]:
x2 = g.V(hkVertexId).out('route').range(0, n).fold().next()
x2

[v[4240], v[4312], v[12320], v[12504], v[16392]]

__Get the results from "n" to the end__

In [12]:
# x1 = g.V(hkVertexId).out('route').range(n, -1).fold().next()
# x1

## skip()
Equivalent to skipping by "n"

In [13]:
# x2 = g.V(hkVertexId).out('route').skip(n).fold().next()
# assert x1 == x2

## tail()
__Get the last "n" results__

In [14]:
g.V(hkVertexId).out('route').tail(n).fold().next()

[v[1790088], v[1794184], v[1806472], v[3383432], v[3395720]]

There is no guarantee to the order of ranges, it is often based on order put into graph

## timeLimit()
Limit the query to however much can be processed within 10 ms

In [15]:
g.V(hkVertexId).repeat(__.timeLimit(10).out()).until(__.has('code','SYD')).path().by('code').next()

['HKG', 'SYD']